# Univariate regression to predict the wind speed field

In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from data_loaders import get_wake_dataloader

In [2]:
DATAFRAME_FILEPATH = "data/wake_dataframe.csv"

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(0)

# hyperparameters
BATCH_SIZE = 256
EPOCHS = 500
LR = 0.0000001

In [3]:
INPUT_VARIABLES = ['x', 'y', 'ws', 'TI', 'ct']
OUTPUT_VARIABLES = ['WS_eff', 'TI_eff', 'P']

#TODO split in training and test sets
train_dataloader = get_wake_dataloader(DATAFRAME_FILEPATH, INPUT_VARIABLES, OUTPUT_VARIABLES, MinMaxScaler(), BATCH_SIZE)
len(train_dataloader)

19872

In [ ]:
input_space = len(INPUT_VARIABLES) # or train_dataloader.dataset.x.shape[1]
output_space = train_dataloader.dataset.y.shape[1]

model = nn.Sequential(
    nn.Linear(input_space, 50),
    nn.ReLU(),
    nn.Linear(50, 250),
    nn.ReLU(),
    nn.Linear(250, output_space),
    nn.ReLU() #TODO?
).to(DEVICE)

optimizer = torch.optim.Adam(
    (p for p in model.parameters() if p.requires_grad), lr=LR
)
loss_function = torch.nn.MSELoss()
model.train()

print("Starting training")
epoch_to_loss_acc = dict()
for epoch in range(EPOCHS):
    epoch_loss = 0
    epoch_accs = torch.FloatTensor()
    for batch in iter(train_dataloader):
        #TODO to device in the dataloader
        x, y = batch

        prediction = model.forward(x)

        # loss and step
        loss = loss_function(prediction, y)
        #optimizer.zero_grad() TODO
        loss.backward()
        #torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5) TODO
        optimizer.step()

        epoch_loss += loss.item()

        #TODO accuracy or other metric computation

    print(f"{epoch=} -> {epoch_loss=}")
    epoch_to_loss_acc[epoch] = epoch_loss

epoch_to_loss_acc

In [ ]:
"""
metrics_df = pd.DataFrame(epoch_to_loss_acc).transpose().reset_index().rename(columns={"index": "Epoch", 0: "Loss", 1: "Speed avg error in wake prediction"})
metrics_df.plot(x="Epoch", y="Loss")
"""

In [ ]:
"""
metrics_df.plot(x="Epoch", y="Speed avg error in wake prediction")
"""